
# ベクトルの可視化

In [1]:
!pip install umap-learn
!pip install plotly
!pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [28]:
import iris 
import umap.umap_ as umap
import numpy as np
import plotly.express as px
import pandas as pd
import plotly.io as pio

In [29]:
# 接続情報
from iris_config import IRIS_USER, IRIS_PASSWORD, IRIS_CONN_STR

# 変換関数：文字列からfloatリストへ変換
def parse_vector_string(s):
    return [float(x) for x in s.split(',')]

# IRISに格納された魚の名前 (もしくは画像)のベクトルを取得
def get_vectors(conn, modality):
    try:
        table = 'FishSearch.Fish' if modality == 1 else 'FishSearch.FishImage'

        sql = f"SELECT Name, Features FROM {table} fish"

        with conn.cursor() as cursor:
            cursor.execute(sql)
            results = cursor.fetchall()
        return results

    except Exception as e:
        return f"Error: {str(e)}"

In [30]:

# IRISに接続
conn = iris.connect(IRIS_CONN_STR, IRIS_USER, IRIS_PASSWORD)

modality = 1 # text
#modality = 2 # image

# IRIS から魚の名前ベクトルを取得
info = get_vectors(conn, modality)

# ベクトルとラベルを分離
labels = [row[0] for row in info]

# ベクトルの変換 (文字列からfloatに変換)
vectors = [parse_vector_string(row[1]) for row in info]

In [31]:
# ３次元のベクトルを可視化 (Plotly)
def visualizeVectors3dInteractive (labels, vectors_3d):  

    #pio.renderers.default = 'notebook_connected'  # または 'iframe_connected', 'vscode', 'browser'
    pio.renderers.default = 'browser' # 別タブで表示
    
    df_3d = pd.DataFrame(vectors_3d, columns=["x", "y", "z"])
    df_3d["label"] = labels
    
    fig = px.scatter_3d(df_3d, x="x", y="y", z="z", text="label")
    fig.update_traces(marker=dict(size=4))
    fig.show()
        

In [32]:
# UMAPを設定
# n_components: 次元数
# n_neighbors: 各点の近傍のサイズ 
## 小さくすると、局所構造が強調され、クラスタが分離されやすい 大きくすると、全体の配置が保存されやすい
# min_dist: 埋め込み空間での最小距離 小さくすると密集し、大きくすると散らばる配置となる
umap_model_3d =  umap.UMAP(n_components=3, n_neighbors=15, min_dist=0.2, metric='cosine')


In [33]:
# テキストベクトルの次元を削減して 3Dに
reduced_3d = umap_model_3d.fit_transform(vectors)


In [34]:
# テキストベクトルの 3D 可視化 (インタラクティブ)
visualizeVectors3dInteractive (labels, reduced_3d)

### イメージ ベクトルの可視化

In [14]:
modality = 2 # image

# IRIS に格納された画像ベクトルを取得
image_info = get_vectors(conn, modality)

# ベクトルとラベルを分離
image_labels = [row[0] for row in image_info]
#vectors = [row[1] for row in info]
# ベクトルの変換 (文字列からfloatに変換)
image_vectors = [parse_vector_string(row[1]) for row in image_info]

In [ ]:
# 次元削減して3次元に
image_reduced_3d = umap_model_3d.fit_transform(image_vectors)

In [16]:
# 画像ベクトルの 3D 可視化 (インタラクティブ)
visualizeVectors3dInteractive (image_labels, image_reduced_3d)

### イメージとテキストのベクトルを同じ空間で可視化

In [11]:
# ベクトルの統合(画像+テキスト) 
multi_modal_vectors = vectors + image_vectors
multi_modal_labels = labels + image_labels

In [ ]:

# UMAPの設定
# n_components: 次元数
# n_neighbors: 各点の近傍のサイズ 
## 小さくすると、局所構造が強調され、クラスタが分離されやすい 大きくすると、全体の配置が保存されやすい
# min_dist: 埋め込み空間での最小距離 小さくすると密集し、大きくすると散らばる配置となる
umap_model_3d =  umap.UMAP(n_components=3, n_neighbors=70, min_dist=0.1, metric='cosine')


# 画像+テキスト ベクトルを次元削減して3次元に
multi_modal_reduced_3d = umap_model_3d.fit_transform(multi_modal_vectors)

In [13]:
# 画像+テキスト ベクトルの 3D 可視化 (インタラクティブ) ※画像とテキストが離れて表示されるのは、モダリティギャップという現象です
visualizeVectors3dInteractive (multi_modal_labels, multi_modal_reduced_3d)

# クラスタリング


In [35]:
from sklearn.cluster import KMeans

# クラスタ数(K)を設定
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=5)
cluster_labels = kmeans.fit_predict(vectors)

In [36]:

def visualizeVectors3dInteractive_with_clusters(labels, vectors_3d, cluster_labels):
    pio.renderers.default = 'browser'

    # データフレーム作成
    df_3d = pd.DataFrame(vectors_3d, columns=["x", "y", "z"])
    df_3d["label"] = labels
    df_3d["cluster"] = cluster_labels

    # 3D散布図（クラスタごとに色分け）
    fig = px.scatter_3d(
        df_3d,
        x="x", y="y", z="z",
        color="cluster",              # クラスタごとに色分け
        text="label",                 # ラベル表示
        symbol="cluster",             # オプション：クラスタごとにマーカーも変える
    )
    
    fig.update_traces(marker=dict(size=5))
    fig.update_layout(title="3D UMAP (with Clusters)", legend_title="Cluster")
    fig.show()

In [37]:
visualizeVectors3dInteractive_with_clusters(labels, reduced_3d, cluster_labels)

In [38]:
# クラスタ数(K)を再設定
n_clusters = 11
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(vectors)

In [39]:
visualizeVectors3dInteractive_with_clusters(labels, reduced_3d, cluster_labels)

### アノマリ検知